# 0. Build Team Identities, Company Employment and User Identities

Notebook to build all of the different identities assumed available when I have initiailised Type Refinery. Note that the Stix objects created by these classes are frozen dicts, so its better to create the components and then the whole, rather than create an empty whole first and then add the components, which requires updating. Note we have added on some system identities, such as Exchange server, and some Laptops

We will take the Type Refinery User as an Example of the Process of making a group of Complex Identity using the official [Identity_Contact_Information Extension](https://github.com/os-threat/cti-stix-common-objects/tree/main/extension-definition-specifications/identity-contact-information) specification, and combining it with the standard Stix approach. There are five stages:

1. Create the `User_Account` first,
2. Then the `Email_Address` objects, attaching the new User_Account.["id"]
3. Then create the `EmailContact`, `SocialMediaContact`, `ContactNumber` lists of sub-objects from the specification, attaching their User_Account.["id"] and Email_Address.["id"] as required
4. Create the [Identity Stix object](https://docs.oasis-open.org/cti/stix/v2.1/os/stix-v2.1-os.html#_wh296fiwpklp) with the [Identity_Contact_Information Extension](https://github.com/os-threat/cti-stix-common-objects/blob/main/extension-definition-specifications/identity-contact-information/Identity%20Contact%20Information.docx), and joining the sub objects to the Extension object and then down to the base Identity object
5. Collect all of the objects and put them into a bundle, and save into a json

Then we will create identities fro the three dummy users that are always clicking the wrong links, but at least they are good on reporting. Finally, we will employ the four Identities with our current company, and save the complete group of objects into a bundle

Each time a block makes an object, and saves it as a json, this notebook will parse the object into an actual Stix object, so it can be bundled and printy printed (It also verifies the objects are created correctly, and is cool).

![](img/Evidence_Identities.png)
![](img/Evidence_Identities_DataFlows.png)

## A. Load Imports

### A.1 StixORM Imports


In [1]:
import sys
!{sys.executable} -m pip install stixorm
from stixorm.module.typedb import TypeDBSink, TypeDBSource
from stixorm.module.authorise import import_type_factory
from stixorm.module.definitions.stix21 import (
    Identity, EmailAddress, UserAccount, Relationship, Bundle
)
from stixorm.module.definitions.os_threat import (
    IdentityContact, EmailContact, SocialMediaContact, ContactNumber
)
from stixorm.module.authorise import import_type_factory
from stixorm.module.typedb_lib.instructions import ResultStatus, Result
from stixorm.module.parsing import parse_objects
import_type = import_type_factory.get_all_imports()
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

### A.2 Relative Path Imports
https://stackoverflow.com/questions/76162459/jupyter-notebook-importerror-attempted-relative-import-with-no-known-parent-pac/77528726#77528726

In [2]:
import sys
sys.path.append('../')
import os

### A.3 Relative Import of Blocks and Phishing Scenario Data

Also the data needed to initialise the company working on the phishing scenario

In [3]:
import json
from Utilities.local_make_general import invoke_context_save_block
from Utilities.util import emulate_ports, unwind_ports, conv
from Utilities.local_make_sdo import invoke_make_identity_block
from Utilities.local_make_sco import invoke_make_user_account_block, invoke_make_email_addr_block
from Utilities.local_make_sro import invoke_sro_block

path_base = "../Block_Families/Objects/"
results_base = "../Orchestration/Results/"
sro_data = "SRO/Relationship/sro_attributed.json"
identities_data = [
    {
        "who": "me", "email": "SCO/Email_Addr/email_addr_TR_user.json", 
        "acct": "SCO/User_Account/usr_account_TR_user.json",
        "ident": "SDO/Identity/identity_TR_user.json",
        "results": "step0/TR_Identity"
    }, 
    {
        "who": "usr1", "email": "SCO/Email_Addr/email_addr_IT_user1.json", 
        "acct": "SCO/User_Account/usr_account_IT_user1.json",
        "ident": "SDO/Identity/identity_IT_user1.json",
        "results": "step0/Usr1"
    }, 
    {
        "who": "usr2", "email": "SCO/Email_Addr/email_addr_IT_user2.json", 
        "acct": "SCO/User_Account/usr_account_IT_user2.json",
        "ident": "SDO/Identity/identity_IT_user2.json",
        "results": "step0/Usr2"
    }, 
    {
        "who": "usr3", "email": "SCO/Email_Addr/email_addr_IT_user3.json", 
        "acct": "SCO/User_Account/usr_account_IT_user3.json",
        "ident": "SDO/Identity/identity_IT_user3.json",
        "results": "step0/Usr3"
    }, 
]
company_base = {
    "ident": "SDO/Identity/identity_TR_user_company.json",
    "results": "step0/employer"
}
systems_base = [
    {
        "data_path": "SDO/Identity/identity_Exchange.json",
        "results": "step0/Systems_Exchange"
    }
]
assets_base = [
    {
        "data_path": "SDO/Identity/identity_Laptop1.json",
        "results": "step0/Systems_Laptop1"
    },
    {
        "data_path": "SDO/Identity/identity_Laptop2.json",
        "results": "step0/Systems_Laptop2"
    }
]
#
# Make the Variables to hold the Context json that gets saved to a Local File
Type_Refinery_Context = {}
Type_Refinery_Context["local"] = {}
local = Type_Refinery_Context["local"]
local["me"] = []
local["team"] = []
local["company"] = []
local["systems"] = []
local["systems"] = []
local["assets"] = []
Type_Refinery_Context["incident"] = {}
incident = Type_Refinery_Context["incident"]
incident["incident_obj"] = {}
incident["sequence_start_objs"] = []
incident["sequence_objs"] = []
incident["task_objs"] = []
incident["event_objs"] = []
incident["impact_objs"] = []
incident["other_object_objs"] = []

Type_Refinery_Context["typedb"] = {}
typedb = Type_Refinery_Context["typedb"]
typedb["incident_obj"] = {}
typedb["sequence_start_objs"] = []
typedb["sequence_objs"] = []
typedb["task_objs"] = []
typedb["event_objs"] = []
typedb["impact_objs"] = []
typedb["other_object_objs"] = []
#
# Set the paths for the Context and the Bundle
TR_identity_bundle_path = "./Context_Mem/identity_bundle.json"
TR_Context_Memory_Path = "./Context_Mem/Type_Refinery_Context.json"
# Make the Variables to hold the Bundle List that gets saved to TypeDB
results_paths = {}

## B. Create the Multi-Faceted User of this Type Refinery

only perform once on initialisation, otherwise you must update the identity object

### B.1 First Select the User Data

In [4]:
for ident in identities_data:
    if ident["who"] == "me":
        email_path = ident["email"]
        acct_path = ident["acct"]
        ident_path = ident["ident"]
        results_path = ident["results"]


### B.2 Make the User Account for the TR User
Simply pass the paths for the input file and where the results should be written

In [5]:
# 1. invoke the account object using method from ./Utilities/local_make_sco.py
TR_user_acct_list = invoke_make_user_account_block(acct_path, results_path)
# 2. add the record to the in-session bundles and context lists
local["me"] = local["me"] + TR_user_acct_list
# recover the threat email object in dict form as a global variable for later
TR_user_account = conv(TR_user_acct_list[0])

{
    "type": "user-account",
    "spec_version": "2.1",
    "id": "user-account--83658594-537d-5c32-b9f0-137354bd9bc3",
    "user_id": "79563902",
    "account_login": "tjones",
    "account_type": "soc,",
    "display_name": "Trusty Jones"
}


### B.3 Make the Email Address for the TR User

In [6]:
# 1. invoke the account object using method from ./Utilities/local_make_sco.py
TR_email_addr_list = invoke_make_email_addr_block(email_path, results_path, TR_user_account)
# 2. add the record to the in-session bundles and context lists
local["me"] = local["me"] + TR_email_addr_list
# recover the threat email object in dict form as a global variable for later
TR_email_addr = conv(TR_email_addr_list[0])

{
    "type": "email-addr",
    "spec_version": "2.1",
    "id": "email-addr--c99b87bd-f0a8-50ca-9f84-68072efc61e3",
    "value": "trusty@example.com",
    "display_name": "Trusty Jones",
    "belongs_to_ref": "user-account--83658594-537d-5c32-b9f0-137354bd9bc3"
}


### B.4 Make the Identity for the TR User

In [7]:
# 1. invoke the account object using method from ./Utilities/local_make_sco.py
TR_ident_list = invoke_make_identity_block(ident_path, results_path, email_results=TR_email_addr_list, acct_results=TR_user_acct_list)
# 2. add the record to the in-session bundles and context lists
local["me"] = local["me"] + TR_ident_list
# recover the threat email object in dict form as a global variable for later
TR_ident = conv(TR_ident_list[0])

{
    "type": "identity",
    "spec_version": "2.1",
    "id": "identity--2e02a834-c89b-483b-869d-fcb713721246",
    "created": "2023-12-31T08:02:45.042563Z",
    "modified": "2023-12-31T08:02:45.042563Z",
    "name": "Trusty Jones",
    "description": "A Trusty Individual",
    "roles": [
        "soc",
        "support"
    ],
    "identity_class": "individual",
    "sectors": [
        "technology"
    ],
    "extensions": {
        "extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017a498": {
            "extension_type": "property-extension",
            "contact_numbers": [
                {
                    "contact_number_type": "work-phone",
                    "contact_number": "0418-208-368"
                }
            ],
            "email_addresses": [
                {
                    "digital_contact_type": "work",
                    "email_address_ref": "email-addr--c99b87bd-f0a8-50ca-9f84-68072efc61e3"
                }
            ],
            "first_nam

## C. Employ the User

only perform once on initialisation, otherwise you must update the identity object

### C.1 First Create the Company

In [8]:
cwd = os.getcwd()
#
# Setup company info into paths
company_ident_path = company_base["ident"]
company_results_path = company_base["results"]
#
# MAke the Company object nd add it to the bundle_list
#
comp_ident_list = invoke_make_identity_block(company_ident_path, company_results_path)
# 2. add the record to the in-session bundles and context lists
local["company"] = local["company"] + comp_ident_list
# recover the threat email object in dict form as a global variable for later
comp_ident = conv(TR_ident_list[0])

{
    "type": "identity",
    "spec_version": "2.1",
    "id": "identity--cf7723f6-d3e3-4633-b67f-11d8f7dec556",
    "created": "2023-12-31T08:02:45.073264Z",
    "modified": "2023-12-31T08:02:45.073264Z",
    "name": "Example Company",
    "description": "An Example Organisation",
    "identity_class": "organization",
    "sectors": [
        "technology"
    ]
}


### C.2 Setup Employment Relationship

Since there is no employed-by SRO setup yet, we will use the attributed-to SRO as a stand-in until we implement it

In [9]:
#
# There is no "employed-by" SRO, instead we use the "attributed-to" SRO
#
employed_list = invoke_sro_block(sro_data, results_path, source=comp_ident, target=TR_ident, relationship_type="attributed-to")
# 2. add the record to the in-session bundles and context lists
local["company"] = local["company"] + employed_list

{
    "type": "relationship",
    "spec_version": "2.1",
    "id": "relationship--9605d305-dc6f-4175-b5cf-ae5fa3b1c32b",
    "created": "2023-12-31T08:02:45.10827Z",
    "modified": "2023-12-31T08:02:45.10827Z",
    "relationship_type": "attributed-to",
    "source_ref": "identity--2e02a834-c89b-483b-869d-fcb713721246",
    "target_ref": "identity--2e02a834-c89b-483b-869d-fcb713721246"
}


## D. Create and Employ the 3 Sales Uses

For each data record in the list, make the user account, the email-addr, identity and employ them

In [10]:
for ident in identities_data:
    results_paths = {}
    if ident["who"][:3] == "usr":
        email_path = ident["email"]
        acct_path = ident["acct"]
        ident_path = ident["ident"]
        results_path = ident["results"]
        # 1. invoke the account object using method from ./Utilities/local_make_sco.py
        user_acct_list = invoke_make_user_account_block(acct_path, results_path)
        user_email_addr_list = invoke_make_email_addr_block(email_path, results_path, user_acct_list[0])
        user_ident_list = invoke_make_identity_block(ident_path, results_path,email_results=user_email_addr_list, acct_results=user_acct_list)
        employed_list = invoke_sro_block(sro_data, results_path, source=comp_ident, target=user_ident_list[0])
        # 2. add the record to the in-session bundles and context lists
        local["company"] = local["company"] + user_acct_list + user_email_addr_list + user_ident_list + employed_list

{
    "type": "user-account",
    "spec_version": "2.1",
    "id": "user-account--597ad4d4-35ba-585d-8f6d-134a75032f9b",
    "user_id": "79563902",
    "account_login": "nsmith",
    "account_type": "sales,",
    "display_name": "Naive Smith"
}
{
    "type": "email-addr",
    "spec_version": "2.1",
    "id": "email-addr--4722424c-7012-56b0-84d5-01d076fc547b",
    "value": "naive@example.com",
    "display_name": "Naive Smith",
    "belongs_to_ref": "user-account--597ad4d4-35ba-585d-8f6d-134a75032f9b"
}
{
    "type": "identity",
    "spec_version": "2.1",
    "id": "identity--e5a6e5c3-7138-4288-8146-87d972eb2098",
    "created": "2023-12-31T08:02:45.156099Z",
    "modified": "2023-12-31T08:02:45.156099Z",
    "name": "Naive Smith",
    "description": "A Naive Individual",
    "roles": [
        "user",
        "sales"
    ],
    "identity_class": "individual",
    "sectors": [
        "technology"
    ],
    "extensions": {
        "extension-definition--66e2492a-bbd3-4be6-88f5-cc91a017

## E. Create and Save the List of IT System Identities


In [11]:
for IT_system in systems_base:
    #
    # Setup company info into paths
    system_ident_path = IT_system["data_path"]
    system_results_path = IT_system["results"]
    #
    # MAke the Company object nd add it to the bundle_list
    #
    system_ident_list = invoke_make_identity_block(company_ident_path, company_results_path)
    # 2. add the record to the in-session bundles and context lists
    local["systems"]  = local["systems"]  + system_ident_list

{
    "type": "identity",
    "spec_version": "2.1",
    "id": "identity--1e322625-0c66-4b03-87e8-7ed69a8fc26e",
    "created": "2023-12-31T08:02:45.277188Z",
    "modified": "2023-12-31T08:02:45.277188Z",
    "name": "Example Company",
    "description": "An Example Organisation",
    "identity_class": "organization",
    "sectors": [
        "technology"
    ]
}


## F. Create and Save the List of IT Hardware Assets


In [12]:
for asset in assets_base:
    #
    # Setup company info into paths
    asset_ident_path = asset["data_path"]
    asset_results_path = asset["results"]
    #
    # MAke the Company object nd add it to the bundle_list
    #
    asset_ident_list = invoke_make_identity_block(asset_ident_path, asset_results_path)
    # 2. add the record to the in-session bundles and context lists
    local["assets"]  = local["assets"]  + asset_ident_list

{
    "type": "identity",
    "spec_version": "2.1",
    "id": "identity--75d7bcb7-8639-45f4-952e-96845b95e109",
    "created": "2023-12-31T08:02:45.296191Z",
    "modified": "2023-12-31T08:02:45.296191Z",
    "name": "HP Laptop 1",
    "description": "Usr 1's Laptop",
    "roles": [
        "laptop"
    ],
    "identity_class": "system",
    "sectors": [
        "technology"
    ],
    "external_references": [
        {
            "source_name": "HP Ultima 10",
            "description": "16GB RAM, 1TB SD, Microsoft Windows 11 Professional",
            "url": "https://mail.echange.microsoft.com",
            "external_id": "Asset--629456a"
        }
    ]
}
{
    "type": "identity",
    "spec_version": "2.1",
    "id": "identity--b2348432-8df1-4245-996b-95e15f787f85",
    "created": "2023-12-31T08:02:45.307121Z",
    "modified": "2023-12-31T08:02:45.307121Z",
    "name": "HP Laptop 2",
    "description": "Usr 2's Laptop",
    "roles": [
        "laptop"
    ],
    "identity_class": 

## F. Create and Save the Type Refinery Context Memory


In [14]:
# Save the Tpe Refinery Context Memory File
invoke_context_save_block(Type_Refinery_Context)